# Forecast verification
This is the main notebook of the study containing all the skill assessments.

* **Most use seasonal averages for verification. How to best aggregate data? Needed in daily resolution for hindcast. Initial timestep of model = 20 mins (correct?)... if I aggregate to seasonal averages I could have just as well downloaded the monthly aggregated data firsthand and not bothered with the large daily dataset... how can daily data be considered in the analysis?**
* see "Effective Sample Size" -> should I use it? see https://doi.org/10.1175/1520-0442(1999)012<1990:TENOSD>2.0.CO;2  Also compute the p values! Understand how this is used in forecast verification!

* Check more verification datasets in one HindcastEnsemble? [Note](https://youtu.be/EcMxImmMBec?t=8526) more than one verification dataset can be carried along! This allows to compare the skill for multiple observational datasets.

# Misc TODO

## Climpred things
* Check: [Note](https://youtu.be/EcMxImmMBec?t=8647) a skill dimension is appended to the HindcastEnsemble object which allows to compare the skills of the forecast product and the skill of the reference forecast (e.g. persistence) -> good plot of the spatial distribution of skill in two rows, one for the forecast product and the other one for the reference product
* --> just not sure how e.g. the ACC is computed then if we are not even in anomaly space. Does climpred actually compute anomalies?
* decide which alignment to use -> literature?


## EOF analysis
* compute the EOFs
    * https://ajdawson.github.io/eofs/latest/
    * read Wilks chapter

## Detrending
* https://climpred.readthedocs.io/en/stable/api/climpred.stats.rm_poly.html#climpred.stats.rm_poly
    * or better in XCLIM or straight away in Xarray?
    * see here: https://youtu.be/SKXUBD6DGao?t=563
* I will probably not need to worry about detrending at seasonal timescales... where is literature which specifies at which time intervals detrending becomes relevant? Do I need to justify?


In [1]:
from climpred import HindcastEnsemble
from pathlib import Path
import xarray as xr

import matplotlib.pyplot as plt
import cmocean.cm as cmo # colormaps

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
shp = Path(r"C:/Users/sb123/Documents/OneDrive/04_SS22/climThesis/Data/Geo/THU.shp")
shape_feature = ShapelyFeature(Reader(shp).geometries(), ccrs.PlateCarree(), facecolor="none", edgecolor="red")

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

C:\Users\sb123\.conda\envs\Thesis2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# read data from netCDFs and set-up a climpred HindcastEnsemble instance
hc = xr.open_dataarray(r"C:\Users\sb123\Documents\Massendaten\hindcast.nc")
obs = xr.open_dataarray(r"C:\Users\sb123\Documents\Massendaten\Observations_fin.nc")
hindcast = HindcastEnsemble(hc)
hindcast = hindcast.add_observations(obs)
del obs, hc

In [8]:
hindcast

<climpred.HindcastEnsemble>
Initialized:
    tp       (init, member, latitude, longitude, lead) float32 1.652 ... 3.519
Uninitialized:
    None
Observations:
    tp       (time, latitude, longitude) float32 2.944 3.973 3.721 ... 0.0 0.0

In [10]:
# Testing only:
# filter for inits on a specific month to compare results against https://meteoswiss-climate.shinyapps.io/skill_metrics/
hindcast.verify(metric="additive_bias", comparison="e2o", dim=[], alignment="same_inits")
# hindcast.where(hindcast.coords["valid_time.season"] == "JJA", drreop=True).where(hindcast.coords["init.month"] == 1, drop=True)

INFO:root:initialized | lead: 1.0 | inits: 1994-01-01 00:00:00-2016-05-01 00:00:00 | verifs: 1994-01-02 00:00:00-2016-05-02 00:00:00
INFO:root:initialized | lead: 2.0 | inits: 1994-01-01 00:00:00-2016-05-01 00:00:00 | verifs: 1994-01-03 00:00:00-2016-05-03 00:00:00
INFO:root:initialized | lead: 3.0 | inits: 1994-01-01 00:00:00-2016-05-01 00:00:00 | verifs: 1994-01-04 00:00:00-2016-05-04 00:00:00
INFO:root:initialized | lead: 4.0 | inits: 1994-01-01 00:00:00-2016-05-01 00:00:00 | verifs: 1994-01-05 00:00:00-2016-05-05 00:00:00
INFO:root:initialized | lead: 5.0 | inits: 1994-01-01 00:00:00-2016-05-01 00:00:00 | verifs: 1994-01-06 00:00:00-2016-05-06 00:00:00
INFO:root:initialized | lead: 6.0 | inits: 1994-01-01 00:00:00-2016-05-01 00:00:00 | verifs: 1994-01-07 00:00:00-2016-05-07 00:00:00
INFO:root:initialized | lead: 7.0 | inits: 1994-01-01 00:00:00-2016-05-01 00:00:00 | verifs: 1994-01-08 00:00:00-2016-05-08 00:00:00
INFO:root:initialized | lead: 8.0 | inits: 1994-01-01 00:00:00-2016-0

<xarray.Dataset>
Dimensions:     (longitude: 11, latitude: 9, lead: 215, init: 269)
Coordinates:
  * longitude   (longitude) float32 5.0 6.0 7.0 8.0 9.0 ... 12.0 13.0 14.0 15.0
  * latitude    (latitude) float32 55.0 54.0 53.0 52.0 51.0 50.0 49.0 48.0 47.0
  * lead        (lead) float64 1.0 2.0 3.0 4.0 5.0 ... 212.0 213.0 214.0 215.0
  * init        (init) object 1994-01-01 00:00:00 ... 2016-05-01 00:00:00
    valid_time  (lead, init) object 1994-01-02 00:00:00 ... 2016-12-02 00:00:00
    skill       <U11 'initialized'
Data variables:
    tp          (lead, init, latitude, longitude) float32 -4.827 ... -0.9449
Attributes:
    prediction_skill_software:     climpred https://climpred.readthedocs.io/
    skill_calculated_by_function:  HindcastEnsemble.verify()
    number_of_members:             25
    alignment:                     same_inits
    metric:                        unconditional_bias
    comparison:                    e2o
    dim:                           []
    reference:                     []

In [ ]:
convert_init_lead_to_valid_time_lead(hindcast.get_initialized())

In [ ]:
# Doing some visualisations and plausibility checks on our climpred
# HindcastEnsemble

# fig, ax = plt.subplots()
data = hindcast.get_observations().tp.values.flat
# import numpy as np
data[np.where(data < 0)[0]]

# sns.histplot(data=hindcast.get_observations().tp.values[:,0,0])

In [ ]:
%time
#
verification_e2o = hindcast.verify(
    metric="pearson_r", comparison="e2o", dim="init", alignment="same_inits"
)

In [ ]:
verification_e2o

**Check, do I still have a member dimension when verifying with m2o?**

**ToDo:** Also calculate the p - values and research the underlying
methodology (see Wilks 2019)

Da läuft noch was schief mit den Verification Alignments

In [ ]:
fig, ax = plt.subplots(
    ncols=2, figsize=(10, 5), subplot_kw={"projection": ccrs.PlateCarree()}
)
verification.sel({"lead": 1})["tp"].plot(ax=ax[0], vmin=0, vmax=0.6)
verification.sel({"lead": 100})["tp"].plot(ax=ax[1], vmin=0, vmax=0.6)

for ax in ax.flatten():
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.COASTLINE)
plt.tight_layout()

In [ ]:
verification.dims

In [ ]:
fig, ax = plt.subplots()
y = verification.tp.sel({"longitude": 5, "latitude": 55}).values
x = verification.lead.values
ax.plot(x, y)
fig.show()

# TODO compute manually and compare the graphs!

In [ ]:
verification.tp.shape

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})
verification.tp.sel({"lead": 30}).plot(ax=ax)
ax.add_feature(cfeature.COASTLINE)
fig.show()

In [ ]:
verification.tp.sel({"lead": 30})